In [1]:
# import & 文件路径
import mne
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from Globals import *
from mne.preprocessing import ICA, create_eog_epochs
import csv
%matplotlib qt


PATH = 'D:\Onedrive\毕业论文\实验数据\实验2数据\pre'

In [2]:
for sub in Sublist_exp2:
    os.chdir(PATH)
    filename = 'Sub_0' + sub
    eegname = '0' + sub + '.vhdr'
    os.chdir(filename)
    raw = mne.io.read_raw_brainvision(eegname, eog = ['EOG'], preload = True)
    raw.set_eeg_reference('average', ch_type = 'eeg')
    raw.set_montage('standard_1020')
    # 将原始文件中的 annotation 读入 raw_annot 中；
    raw_annot = raw.annotations
    # make_relax_annotation 函数将 marker 之间间隔超过4.5秒的数据作为休息时间进行标记；
    bad_relax_annot = make_relax_annotations(raw)   
    #将休息时间与原始的 annotation 合并；
    raw.set_annotations(raw_annot + bad_relax_annot)
    #crop_raw_relax函数对实验末尾的休息时间进行标记；
    crop_raw_relax(raw)
    if sub in exp2_bad.keys():
        if sub == '36':
            pass
        else:
            print(sub)
            print('is_bad')
            raw.info['bads'].append(exp2_bad[sub])
            raw.interpolate_bads()
    # 复制raw数据为 raw_for_ica 用于 ica; ica 数据进行 1,40 带通
    raw_for_ica = raw.copy().filter(1.,40)
    # 对原始数据进行 0.1,40 带通（用于实际分段）
    raw.filter(0.1,40)
    # 对 raw_for_ica 实例进行 ica 运算；
    print('now in' + sub + '/8')
    ica = mne.preprocessing.ICA(n_components = 0.999999,method = 'picard', max_iter= 1000, random_state= 42)
    ica.fit(raw_for_ica)
    #eog_epochs = create_eog_epochs(raw_for_ica)
    # 根据ica结果和相关系数来找出与EOG活动相关的ICA成分, threshold参数设定判断outlier的Z-socre
    eog_inds, scores = ica.find_bads_eog(raw_for_ica, threshold= 2)
    # 返回 eog_inds 为对应 ica 成分的index
    
    mne.preprocessing.ICA.save(ica, fname = 'ica-ica.fif', overwrite = True)



Extracting parameters from 01.vhdr...
Setting channel info structure...
Reading 0 ... 2566679  =      0.000 ...  2566.679 secs...
Applying average reference.
Applying a custom ('EEG',) reference.
Used Annotations descriptions: ['Comment/actiCAP Data On', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 13', 'Stimulus/S 14', 'Stimulus/S 15', 'Stimulus/S 16', 'Stimulus/S 17', 'Stimulus/S 18', 'Stimulus/S 19', 'Stimulus/S 22', 'Stimulus/S 33']
Used Annotations descriptions: ['Comment/actiCAP Data On', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 13', 'Stimulus/S 14', 'Stimulus/S 15', 'Stimulus/S 16', 'Stimulus/S 17', 'Stimulus/S 18', 'Stimulus/S 19', 'Stimulus/S 22', 'Stimulus/S 33']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a o